<a href="https://colab.research.google.com/github/kahramankaya/Anlas-l-rEkonomiYoutube/blob/main/79.video(Repeated%20Measures%20ANOVA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#79.video(Repeated Measures ANOVA)

In [25]:
import pandas as pd
import numpy as np
data=pd.DataFrame(
{
"kısıler":["A","B","C","D","X","Y"],
"testoncesı":np.random.randint(1000,size=(6)) ,  
"testsonrası1":np.random.randint(1000,size=(6)),
"testsonrası2":np.random.randint(1000,size=(6)),
"testsonrası3":np.random.randint(1000,size=(6)),
}
)
print(data)

  kısıler  testoncesı  testsonrası1  testsonrası2  testsonrası3
0       A         699           729           262           416
1       B         528           665           368           159
2       C         102           723           981           197
3       D         336           728           217           944
4       X         773           510           991           825
5       Y         906           723           867           116


In [38]:
#pandas melt ile bu verıyı duzgun hale getırelım.
pd =pd.melt(data , id_vars=["kısıler"], value_vars=["testoncesı","testsonrası1","testsonrası2","testsonrası3"])
pd.columns=["kısıler","test","olcum"]
print(pd)

TypeError: ignored

In [27]:
pd = pd.melt(data, id_vars="kısıler", value_vars=["testoncesı", "testsonrası1", "testsonrası2", "testsonrası3"])
pd.columns = ["kısıler", "test", "olcum"]
print(pd)


   kısıler          test  olcum
0        A    testoncesı    699
1        B    testoncesı    528
2        C    testoncesı    102
3        D    testoncesı    336
4        X    testoncesı    773
5        Y    testoncesı    906
6        A  testsonrası1    729
7        B  testsonrası1    665
8        C  testsonrası1    723
9        D  testsonrası1    728
10       X  testsonrası1    510
11       Y  testsonrası1    723
12       A  testsonrası2    262
13       B  testsonrası2    368
14       C  testsonrası2    981
15       D  testsonrası2    217
16       X  testsonrası2    991
17       Y  testsonrası2    867
18       A  testsonrası3    416
19       B  testsonrası3    159
20       C  testsonrası3    197
21       D  testsonrası3    944
22       X  testsonrası3    825
23       Y  testsonrası3    116
